# Lakeflow Connection

## Context and Requirements

- **Training Day**: Day 2 - Delta Lake & Lakehouse
- **Notebook Type**: Demo
- **Technical Requirements**:
  - Databricks Runtime 16.4 LTS or newer (recommended: 17.3 LTS)
  - Unity Catalog enabled
  - Permissions: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
  - Cluster: Standard or **Serverless Compute** (recommended)
- **Dependencies**: Completed notebook `00_setup.ipynb`
- **Duration**: ~60 minutes

> **Note (2025):** Serverless Compute is now the default mode for new workloads.

## Theoretical Introduction

### Lakeflow - Data Ingestion Platform

**Lakeflow** is Databricks' integrated platform for data flow management:

| Component | Description | Use Case |
|-----------|-------------|----------|
| **COPY INTO** | Batch loading from cloud storage | Scheduled ETL, large files |
| **Auto Loader** | Streaming from cloud storage | Continuous ingestion, small files |
| **Lakeflow Connect** | Managed connectors to SaaS | Salesforce, SAP, Workday |
| **Lakeflow Pipelines** | Declarative ETL (DLT) | Bronze → Silver → Gold |

---

### Batch vs Streaming - Decision Matrix

| Feature | Batch (COPY INTO) | Streaming (Auto Loader) |
|---------|-------------------|-------------------------|
| **Latency** | Minutes-Hours | Seconds-Minutes |
| **File size** | Large (>1GB) | Small (<100MB) |
| **Frequency** | Scheduled (hourly/daily) | Continuous |
| **Cost** | Lower (on-demand) | Higher (always-on) |
| **Complexity** | Low | Medium |
| **Idempotency** | ✅ Built-in | ✅ Checkpoint-based |
| **Schema evolution** | Manual | Automatic |

**When Batch (COPY INTO):**
- ✅ Data arrives in scheduled intervals
- ✅ Large files (> 1GB)
- ✅ Latency is not critical
- ✅ Lower cost requirement

**When Streaming (Auto Loader):**
- ✅ Data arrives continuously (< 1h intervals)
- ✅ Need low latency (< 5 min)
- ✅ Small files (< 100MB each)
- ✅ Real-time dashboards/analytics

## Per-user Isolation

In [0]:
%run ../00_setup

## Configuration

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import datetime
import time

**Paths and variables configuration:**

In [0]:
# Set default catalog and schema
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

# === BATCH DATA PATHS ===
CUSTOMERS_CSV = f"{DATASET_BASE_PATH}/customers/customers.csv"
ORDERS_JSON = f"{DATASET_BASE_PATH}/orders/orders_batch.json"
PRODUCTS_PARQUET = f"{DATASET_BASE_PATH}/products/products.parquet"

# === STREAMING DATA PATHS ===
STREAMING_SOURCE_PATH = f"{DATASET_BASE_PATH}/orders/stream"

# === TECHNICAL PATHS ===
CHECKPOINT_BASE_PATH = f"{DATASET_BASE_PATH}/{raw_user}/lakeflow_checkpoints"
SCHEMA_BASE_PATH = f"{DATASET_BASE_PATH}/{raw_user}/lakeflow_schemas"
BAD_RECORDS_PATH = f"{DATASET_BASE_PATH}/{raw_user}/bad_records"

**Cleanup from previous runs (for demo):**

In [0]:
try:
    dbutils.fs.rm(CHECKPOINT_BASE_PATH, True)
    dbutils.fs.rm(SCHEMA_BASE_PATH, True)
    dbutils.fs.rm(BAD_RECORDS_PATH, True)
except:
    pass

**Configuration verification:**

In [0]:
display(
    spark.createDataFrame([
        ("CATALOG", CATALOG),
        ("BRONZE_SCHEMA", BRONZE_SCHEMA),
        ("SILVER_SCHEMA", SILVER_SCHEMA),
        ("USER", raw_user),
        ("CUSTOMERS_CSV", CUSTOMERS_CSV),
        ("STREAMING_SOURCE_PATH", STREAMING_SOURCE_PATH)
    ], ["Variable", "Value"])
)

---

## Section 1: COPY INTO - Batch Loading

**COPY INTO** is the recommended batch ingestion method:
- **Idempotency**: Automatic tracking of processed files
- **File tracking**: Only new files are loaded on re-run
- **Format support**: CSV, JSON, Parquet, Avro, ORC

### Example 1.1: COPY INTO from CSV

In [0]:
TABLE_CUSTOMERS = f"{BRONZE_SCHEMA}.customers_batch"

**Creating target table:**

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {TABLE_CUSTOMERS} (
  customer_id STRING,
  first_name STRING,
  last_name STRING,
  email STRING,
  phone STRING,
  city STRING,
  state STRING,
  country STRING,
  registration_date DATE,
  customer_segment STRING,
  _ingestion_timestamp TIMESTAMP
) USING DELTA
COMMENT 'Customers data - Bronze layer'
""")

**Execute COPY INTO:**

In [0]:
result = spark.sql(f"""
COPY INTO {TABLE_CUSTOMERS}
FROM (
  SELECT 
    customer_id,
    first_name,
    last_name,
    email,
    phone,
    city,
    state,
    country,
    TO_DATE(registration_date, 'yyyy-MM-dd') as registration_date,
    customer_segment,
    current_timestamp() as _ingestion_timestamp
  FROM '{CUSTOMERS_CSV}'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'false')
COPY_OPTIONS ('mergeSchema' = 'true')
""")

display(result)

**Results verification:**

In [0]:
display(
    spark.createDataFrame([
        ("Table", TABLE_CUSTOMERS),
        ("Records", str(spark.table(TABLE_CUSTOMERS).count()))
    ], ["Metric", "Value"])
)

**Sample data:**

In [0]:
display(spark.table(TABLE_CUSTOMERS).limit(5))

### Example 1.2: Idempotency Test

Running COPY INTO again will **not** add duplicates:

In [0]:
count_before = spark.table(TABLE_CUSTOMERS).count()

# Re-run COPY INTO
spark.sql(f"""
COPY INTO {TABLE_CUSTOMERS}
FROM (
  SELECT 
    customer_id, first_name, last_name, email, phone,
    city, state, country,
    TO_DATE(registration_date, 'yyyy-MM-dd') as registration_date,
    customer_segment,
    current_timestamp() as _ingestion_timestamp
  FROM '{CUSTOMERS_CSV}'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true', 'inferSchema' = 'false')
""")

count_after = spark.table(TABLE_CUSTOMERS).count()

**Comparison:**

In [0]:
display(
    spark.createDataFrame([
        ("Before", count_before),
        ("After", count_after),
        ("Difference", count_after - count_before)
    ], ["State", "Count"])
)

---

## Section 2: Auto Loader - Streaming Ingestion

**Auto Loader (cloudFiles)** is a Databricks-managed streaming source:
- Automatic file discovery (file notifications)
- Incremental processing (only new files)
- Schema inference & evolution
- Checkpoint management
- Scales to millions of files

### Example 2.1: Auto Loader with `availableNow` trigger

---

## Trigger Modes

**Trigger** determines how often streaming query executes micro-batches:

| Mode | Behavior | Use Case |
|------|----------|----------|
| `availableNow=True` | Process everything → stop | Scheduled jobs ⭐ |
| `once=True` | Legacy (deprecated) | - |
| `processingTime="10 second"` | Every 10 seconds | Real-time |
| `continuous="1 second"` | Ultra-low latency | Experimental |

In [0]:
TARGET_TABLE_AL = f"{BRONZE_SCHEMA}.orders_autoloader"
CHECKPOINT_AL = f"{CHECKPOINT_BASE_PATH}/autoloader"
SCHEMA_AL = f"{SCHEMA_BASE_PATH}/autoloader"

spark.sql(f"DROP TABLE IF EXISTS {TARGET_TABLE_AL}")

**Auto Loader readStream configuration:**

In [0]:
df_autoloader = (spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("cloudFiles.schemaLocation", SCHEMA_AL)
    .option("cloudFiles.inferColumnTypes", "true")
    .option("cloudFiles.schemaEvolutionMode", "addNewColumns")
    .load(STREAMING_SOURCE_PATH)
)

**Adding metadata columns:**

In [0]:
df_enriched = (df_autoloader
    .withColumn("_processing_time", F.current_timestamp())
    .withColumn("_source_file", F.input_file_name())
)

**Start streaming with `availableNow` trigger:**

> `availableNow` - processes all available data and stops (batch-like streaming)

In [0]:
query = (df_enriched.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", CHECKPOINT_AL)
    .trigger(availableNow=True)
    .toTable(TARGET_TABLE_AL)
)

query.awaitTermination()

**Auto Loader results:**

In [0]:
display(
    spark.createDataFrame([
        ("Records Loaded", str(spark.table(TARGET_TABLE_AL).count())),
        ("Source Files", str(spark.table(TARGET_TABLE_AL).select("_source_file").distinct().count()))
    ], ["Metric", "Value"])
)

**Inferred schema:**

In [0]:
spark.table(TARGET_TABLE_AL).printSchema()

**Sample data:**

In [0]:
display(spark.table(TARGET_TABLE_AL).limit(10))

---

## Section 3: Rescued Data Column

**Rescued Data Column** is an Auto Loader mechanism for handling unexpected data:

| Scenario | Behavior |
|----------|----------|
| New columns | Saved in `_rescued_data` |
| Type mismatches | Saved in `_rescued_data` |
| Malformed records | Saved in `_rescued_data` |

**Configuration:**
```python
.option("cloudFiles.schemaEvolutionMode", "rescue")
```

**schemaEvolutionMode options:**
- `addNewColumns`: Automatically adds new columns
- `rescue`: New columns → `_rescued_data` JSON
- `failOnNewColumns`: Fail if schema changes
- `none`: Ignores new columns (risky!)

### Example 3.1: Auto Loader with rescue mode

In [0]:
TARGET_TABLE_RESCUE = f"{BRONZE_SCHEMA}.orders_rescued"
CHECKPOINT_RESCUE = f"{CHECKPOINT_BASE_PATH}/rescue"
SCHEMA_RESCUE = f"{SCHEMA_BASE_PATH}/rescue"

spark.sql(f"DROP TABLE IF EXISTS {TARGET_TABLE_RESCUE}")

**Define explicit schema (partial):**

In [0]:
# Deliberately define only some columns - rest will go to _rescued_data
partial_schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("total_amount", DoubleType(), True)
])

**Auto Loader with rescue mode:**

In [0]:
df_rescue = (spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("cloudFiles.schemaLocation", SCHEMA_RESCUE)
    .option("cloudFiles.schemaEvolutionMode", "rescue")  # Rescue mode!
    .schema(partial_schema)  # Partial schema
    .load(STREAMING_SOURCE_PATH)
)

**Start stream:**

In [0]:
query_rescue = (df_rescue.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", CHECKPOINT_RESCUE)
    .trigger(availableNow=True)
    .toTable(TARGET_TABLE_RESCUE)
)

query_rescue.awaitTermination()

**Schema with `_rescued_data` column:**

In [0]:
spark.table(TARGET_TABLE_RESCUE).printSchema()

**Data with rescued columns:**

In [0]:
display(
    spark.table(TARGET_TABLE_RESCUE)
    .limit(5)
)

---

## Section 4: Error Handling

**Error handling strategies in COPY INTO:**

| Mode | Behavior |
|------|----------|
| `PERMISSIVE` | Parses what it can, errors → `_corrupt_record` |
| `DROPMALFORMED` | Removes malformed records |
| `FAILFAST` | Stops on first error |

**`badRecordsPath`** - saves bad records to a folder for later analysis.

### Example 4.1: Error handling with badRecordsPath

In [0]:
TABLE_ERRORS = f"{BRONZE_SCHEMA}.customers_with_validation"

**Creating table with `_corrupt_record`:**

In [0]:
spark.sql(f"DROP TABLE IF EXISTS {TABLE_ERRORS}")

spark.sql(f"""
CREATE TABLE {TABLE_ERRORS} (
  customer_id STRING,
  first_name STRING,
  last_name STRING,
  email STRING,
  phone STRING,
  city STRING,
  state STRING,
  country STRING,
  registration_date STRING,
  customer_segment STRING,
  _corrupt_record STRING,
  _ingestion_timestamp TIMESTAMP
) USING DELTA
""")

**Loading with error handling:**

In [0]:
df_with_errors = (
    spark.read
    .format("csv")
    .option("header", "true")
    .option("columnNameOfCorruptRecord", "_corrupt_record")
    .option("badRecordsPath", BAD_RECORDS_PATH)
    .schema("""
        customer_id STRING,
        first_name STRING,
        last_name STRING,
        email STRING,
        phone STRING,
        city STRING,
        state STRING,
        country STRING,
        registration_date DATE,
        customer_segment STRING,
        _corrupt_record STRING
    """)
    .load(CUSTOMERS_CSV)
    .withColumn("_ingestion_timestamp", F.current_timestamp())
)

#df_with_errors.write.mode("overwrite").saveAsTable(TABLE_ERRORS)

In [0]:
display(df_with_errors)

**Bad records statistics:**

In [0]:
%skip
dbutils.fs.ls(BAD_RECORDS_PATH)

In [0]:
%skip
df_bad_records = spark.read.format("json").load("/Volumes/ecommerce_platform_trainer/default/kion_datasets/trainer/bad_records/20251125T115221/bad_records/")

display(df_bad_records)

---

## Section 5: Lakeflow Connect (Informational)

**Lakeflow Connect** is managed SaaS integration without writing code:

### Supported sources:
- Salesforce
- Workday
- Google Analytics
- HubSpot
- Stripe
- SAP
- Netsuite
- ServiceNow

### Key features:

| Feature | Description |
|---------|-------------|
| **Zero-code** | Configuration via UI |
| **Managed** | Automatic scaling |
| **CDC Support** | Change Data Capture |
| **Schema Evolution** | Automatic updates |
| **Unity Catalog** | Full integration |

### How to start:
1. Workspace → **Data** → **Lakeflow** → **Connect**
2. Choose connector (e.g. Salesforce)
3. Provide credentials
4. Select objects to synchronize
5. Set schedule

### Ingestion methods comparison:

| Method | Use Case |
|--------|----------|
| **COPY INTO** | Files in cloud storage (batch) |
| **Auto Loader** | Files in cloud storage (streaming) |
| **Lakeflow Connect** | Data from SaaS systems |
| **Lakeflow Pipelines** | Transformations Bronze → Silver → Gold |

---

## Summary

### What we achieved:

✅ **COPY INTO** - Idempotent batch loading from CSV/JSON/Parquet

✅ **Auto Loader** - Streaming ingestion with file notifications

✅ **Rescued Data Column** - Handling unexpected data

✅ **Error Handling** - badRecordsPath and corrupt record handling

✅ **Lakeflow Connect** - Integration with SaaS systems

✅ **Trigger Modes** - availableNow vs processingTime

### Key Takeaways:

| # | Rule |
|---|------|
| 1 | **COPY INTO** for batch, **Auto Loader** for streaming |
| 2 | Always use **checkpointLocation** for streaming |
| 3 | **Rescue mode** for safe schema change handling |
| 4 | **availableNow** for scheduled jobs (cost-effective) |
| 5 | **Lakeflow Connect** for SaaS integration (zero-code) |

### Additional Resources:
- [Auto Loader Documentation](https://docs.databricks.com/ingestion/auto-loader/index.html)
- [COPY INTO Reference](https://docs.databricks.com/sql/language-manual/delta-copy-into.html)
- [Lakeflow Connect](https://docs.databricks.com/integrations/ingestion/lakeflow-connect/index.html)

---

## Cleanup (Optional)

In [0]:
# List of created tables
created_tables = [
    "customers_batch",
    "orders_autoloader",
    "orders_rescued",
    "customers_with_validation",
    "orders_trigger_test"
]

**Created resources verification:**

In [0]:
results = []
for table in created_tables:
    full_table = f"{CATALOG}.{BRONZE_SCHEMA}.{table}"
    try:
        if spark.catalog.tableExists(full_table):
            count = spark.table(full_table).count()
            results.append((table, "EXISTS", str(count)))
        else:
            results.append((table, "NOT FOUND", "-"))
    except Exception as e:
        results.append((table, "ERROR", str(e)[:30]))

display(spark.createDataFrame(results, ["Table", "Status", "Records"]))

In [0]:
# Flaga cleanup
CLEANUP_ENABLED = False

**Execute cleanup (if enabled):**
> 💡 *Recommended: Keep data for subsequent notebooks!*

In [0]:
if CLEANUP_ENABLED:
    results = []
    for table in created_tables:
        full_table = f"{CATALOG}.{BRONZE_SCHEMA}.{table}"
        try:
            spark.sql(f"DROP TABLE IF EXISTS {full_table}")
            results.append((table, "DROPPED"))
        except Exception as e:
            results.append((table, f"ERROR: {str(e)[:30]}"))
    
    # Cleanup checkpoints
    try:
        dbutils.fs.rm(CHECKPOINT_BASE_PATH, True)
        results.append(("checkpoints", "REMOVED"))
    except:
        results.append(("checkpoints", "NOT FOUND"))
    
    display(spark.createDataFrame(results, ["Resource", "Status"]))
else:
    display(spark.createDataFrame([
        ("CLEANUP_ENABLED", "False"),
        ("Action", "Change to True to delete resources")
    ], ["Setting", "Value"]))